In [9]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [ ]:
VERSION = "3"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [11]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {
    "RMSprop": tf.keras.optimizers.legacy.RMSprop(),
}

In [13]:
LOSS = {
    "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy(),
    "BinaryFocalCrossentropy": tf.keras.losses.BinaryFocalCrossentropy(
        alpha=0.25, gamma=2.0
    ),
    "SigmoidFocalCrossEntropy": tfa.losses.SigmoidFocalCrossEntropy(
        alpha=0.25, gamma=2.0
    ),
    "FocalLoss": models.FocalLoss(alpha=0.25, gamma=2.0),
}

In [14]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [15]:
BATCH_SIZE = 64

In [ ]:
SUBVERSION = 0
for LOSS_NAME, _ in LOSS.items():
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["RMSprop"],
        loss=LOSS[LOSS_NAME],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-02-28 23:27:59.884044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.3146 - accuracy: 0.8630 - precision: 0.8810 - recall: 0.9418 - auc: 0.9192

2025-02-28 23:28:47.433437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 60s 207ms/step - loss: 0.3146 - accuracy: 0.8630 - precision: 0.8810 - recall: 0.9418 - auc: 0.9192 - val_loss: 0.2408 - val_accuracy: 0.9069 - val_precision: 0.9154 - val_recall: 0.9468 - val_auc: 0.9641
Epoch 2/10
280/280 [==============================] - 57s 202ms/step - loss: 0.2362 - accuracy: 0.9010 - precision: 0.9245 - recall: 0.9430 - auc: 0.9565 - val_loss: 0.2163 - val_accuracy: 0.9146 - val_precision: 0.9297 - val_recall: 0.9422 - val_auc: 0.9701
Epoch 3/10
280/280 [==============================] - 57s 202ms/step - loss: 0.2194 - accuracy: 0.9071 - precision: 0.9311 - recall: 0.9442 - auc: 0.9625 - val_loss: 0.2070 - val_accuracy: 0.9174 - val_precision: 0.9285 - val_recall: 0.9482 - val_auc: 0.9725
Epoch 4/10
280/280 [==============================] - 57s 202ms/step - loss: 0.2095 - accuracy: 0.9123 - precision: 0.9352 - recall: 0.9469 - auc: 0.9658 - val_loss: 0.2006 - val_accuracy: 0.9202 - val_precision: 0.9312 - val_recall: 

...

280/280 [==============================] - 58s 205ms/step - loss: 0.0340 - accuracy: 0.8036 - precision: 0.8452 - recall: 0.8921 - auc: 0.8357 - val_loss: 0.0220 - val_accuracy: 0.8483 - val_precision: 0.8693 - val_recall: 0.9073 - val_auc: 0.9210
Epoch 2/10
280/280 [==============================] - 57s 204ms/step - loss: 0.0213 - accuracy: 0.8489 - precision: 0.8861 - recall: 0.9128 - auc: 0.9118 - val_loss: 0.0189 - val_accuracy: 0.8724 - val_precision: 0.8857 - val_recall: 0.9267 - val_auc: 0.9429
Epoch 3/10
280/280 [==============================] - 57s 204ms/step - loss: 0.0188 - accuracy: 0.8703 - precision: 0.9032 - recall: 0.9234 - auc: 0.9324 - val_loss: 0.0173 - val_accuracy: 0.8829 - val_precision: 0.9080 - val_recall: 0.9157 - val_auc: 0.9518
Epoch 4/10
280/280 [==============================] - 56s 201ms/step - loss: 0.0176 - accuracy: 0.8794 - precision: 0.9115 - recall: 0.9268 - auc: 0.9416 - val_loss: 0.0165 - val_accuracy: 0.8906 - val_precision: 0.9148 - val_recall: 

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV2/v3b/v3b.3/assets


67/67 [==============================] - 12s 171ms/step - loss: 0.0139 - accuracy: 0.9081 - precision: 0.9293 - recall: 0.9319 - auc: 0.9698


2025-03-01 00:07:27.701505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 11s 161ms/step



